In [ ]:
!pip install albumentations
!pip install opencv-python
!pip install ultralytics
!pip install roboflow

In [1]:
import os, shutil
from os import listdir
from os.path import isfile, join
import cv2
import albumentations as A
import copy
import random
import numpy as np
from ultralytics import YOLO
import torch
import shutil

In [15]:
# If you don't have yolov9 set up
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 325 (delta 152), reused 118 (delta 118), pack-reused 123
Receiving objects: 100% (325/325), 2.26 MiB | 24.00 KiB/s, done.
Resolving deltas: 100% (161/161), done.
/home/potenza/Ortho-AUV/vision/yolov9


In [18]:
%cd yolov9

/home/potenza/Ortho-AUV/vision/yolov9


In [21]:
from roboflow import Roboflow

rf = Roboflow(api_key="HV9jB1dcJZHZd1YO5S7C") 
project = rf.workspace("comp400").project("comp400-fish-detection")
dataset = project.version(2).download("yolov9")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov9 in progress : 85.0%
Version export complete for yolov9 format



Extracting Dataset Version Zip to COMP400-Fish-detection-2 in yolov9:: 100%|██████████| 22/22 [00:00<00:00, 11098.71it/s]


In [19]:
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [24]:
!python3 train.py \
     --batch 16 --epochs 20 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
     --data COMP400-Fish-detection-2/data.yaml \
     --weights weights/gelan-c.pt \
     --cfg models/detect/gelan-c.yaml \
     --hyp hyp.scratch-high.yaml \
     --device cpu

train: weights=weights/gelan-c.pt, cfg=models/detect/gelan-c.yaml, data=COMP400-Fish-detection-2/data.yaml, hyp=hyp.scratch-high.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=cpu, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=15, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5 🚀 1e33dbb Python-3.8.10 torch-2.2.1+cu121 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, cls_pw=1.0, dfl=1.5, obj_pw=1.0, iou_t=0.2, anchor_t=5.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=